In [ ]:
import httpx


endpoint = "http://127.0.0.1:8000/generate"

In [ ]:
prompt = "A photo of TOK adult man dressed up for a professional photo shoot"
res = httpx.post(
    endpoint, 
    json={
        "prompt": prompt
    })

res.status_code, res.json()

In [ ]:
# Test api rate limit

endpoint = "http://127.0.0.1:8000/"
res = httpx.get(endpoint)
res.status_code, res.json(), res.headers.get("Retry-After")

In [ ]:
# create token

# import secrets

# print(secrets.token_urlsafe(32))

In [ ]:
import httpx
from decouple import config
import random

API_ACCESS_KEY = config("API_ACCESS_KEY")

headers = {
    "X-API-Key": API_ACCESS_KEY
}

endpoint = "http://127.0.0.1:8000/"
res = httpx.get(endpoint, headers=headers)
res.status_code, res.json(), res.headers.get("Retry-After")

: 

In [ ]:
API_ACCESS_KEY = config("API_ACCESS_KEY")

headers = {
    "X-API-Key": API_ACCESS_KEY
}

endpoint = "http://127.0.0.1:8000/predictions"
res = httpx.get(endpoint, headers=headers)
preds_json = res.json()
res.status_code, res.json(), res.headers.get("Retry-After")

In [ ]:
preds_json[0].get('url')

In [ ]:
# endpoint = "http://127.0.0.1:8000/predictions/qyw56n23adrme0cn24vat9nrjr"
recent_url = f"http://127.0.0.1:8000{preds_json[0].get('url')}"
res = httpx.get(recent_url, headers=headers)
res.status_code, res.json(), res.headers.get("Retry-After")

In [ ]:
from pathlib import Path

for obj in preds_json[:10]:
    path = obj.get("url")
    endpoint = f"http://127.0.0.1:8000{path}"
    res = httpx.get(endpoint, headers=headers)
    if res.status_code not in range(200, 299):
        continue

    data = res.json()
    _id = data.get("id")
    _input = data.get("input")
    num_outputs = _input.get("num_outputs") or 0
    _output = data.get("output")
    if _output is None:
        continue
    output_names = [Path(x).name for x in _output]
    print(_id, num_outputs, _output)

In [ ]:
recent_url = f"http://127.0.0.1:8000/predictions"
res = httpx.get(recent_url, params={"status": "processing"}, headers=headers)
res.status_code, res.json(), res.headers.get("Retry-After")

In [ ]:
from pathlib import Path

BASE_URL = "http://127.0.0.1:8000"

for obj in preds_json[:10]:
    path = obj.get("url")
    endpoint = f"{BASE_URL}{path}"
    res = httpx.get(endpoint, headers=headers)
    if res.status_code not in range(200, 299):
        continue

    data = res.json()
    files = data.get("files") or None
    if files is None:
        continue
    print(data)
    obj_id = data.get("id")
    with httpx.Client() as client:
        for i, file_path in enumerate(files):
            fname = Path(file_path).name
            outpath = GENERATED_DIR / obj_id / fname
            outpath.parent.mkdir(exist_ok=True, parents=True)
            if outpath.exists():
                continue
            url = f"{BASE_URL}{file_path}"
            res = client.get(url, headers=headers)
            res.raise_for_status()
            with open(outpath, "wb") as f:
                f.write(res.content)
    break